In [ ]:
import numpy as np
from numpy.random import randint, uniform, choice

In [ ]:
def generate_population(size: int, capacity: list):
    population = []
    for _ in range(size):
        population.append([uniform(1, x*0.95) for x in capacity])
    return np.array(population)

In [ ]:
def fitness_inga(capacity: np.array, flow: np.array):
    sigma = flow.sum()
    temp = (flow/(capacity-flow)).sum()
    T = temp/sigma
    return 1/T

def gautil(flow: np.array, capacity: np.array):
    return flow/capacity


In [ ]:
def tournament_selection(size: int, pop: np.array, fitness: np.array, k=5):
    population = list(pop)
    select = []
    for _ in range(size):
        selection_ix = randint(len(population))
        for ix in np.random.randint(0, len(population), k-1):
            if fitness[ix] > fitness[selection_ix]:
                selection_ix = ix
        select.append(population.pop(selection_ix))
    return np.array(select)

In [ ]:
def scatter_crossover(parent1: np.array, parent2: np.array, alpha: int):
    c1 = np.copy(parent1)
    c2 = np.copy(parent2)

    n_genes = len(parent1)
    scatter_point = choice(n_genes, size=int(alpha * n_genes), replace=False)

    for i in scatter_point:
        c1[i] = parent2[i]
        c2[i] = parent1[i]

    return c1, c2

In [ ]:
def innerga(size=10, generasi=100, capacity=None, alpha=0.5, pMu=0.2, pCr=0.8):
    pop = generate_population(size, capacity)
    # print(pop)

    for gen in range(generasi):
        fitness = np.array([fitness_inga(capacity=capacity, flow=ind) for ind in pop])

        selected = tournament_selection(size=size,pop=pop, fitness=fitness)

        offspring = []
        for p1, p2 in zip(selected[::2], selected[1::2]):
            if cross := scatter_crossover(alpha=alpha, parent1=p1, parent2=p2, indpb=pCr):
                c1, c2 = cross
                offspring.append(c1)
                offspring.append(c2)
            else:
                offspring.append(p1)
                offspring.append(p2)

        for m in list(offspring):
            mut = mutGaussian(m, 0, 1, pMu)
            offspring.append(mut)

    return pop, np.array([fitness_inga(capacity=capacity, flow=ind) for ind in pop])
